In [ ]:
# Importing Libraries Required to import API's and Graph/Interface with Data:
import os
import numpy as np
import pandas as pd
import json as js
import requests
import http.client
from dotenv import load_dotenv
from pathlib import Path
from tkinter import *
from tkinter import ttk

In [ ]:
# Loading .env environment variable:
load_dotenv()

#### Extracting zip data and property data by Top 5 investment cities to be used in drop down list

In [ ]:
# Defining csv paths to read Top Zip data and prop_type data csv which includes zip code and property type data:
zip_csv = Path("../Data/zip_data.csv")

# imorting zip.csv and prop.csv to incorporate data into API, data selections:
zip_df = pd.read_csv(zip_csv)
zip_df.head()

In [ ]:
# setting zip_string, list variable and creating zip_list which includes zip info from df above:
zip_list = zip_df['Zip Codes'].tolist()

In [ ]:
# building out dropdown of zip codes for user to make a selection and store in variable:
# creating Tkinter window:
master = Tk()
master.title("Select a Zip Code")
master.geometry("300x200")

# defining function that will store zip selection once button is clicked:
def selected():
    zips = myselect.get()
    print(zips)
    
    global zip_var
    zip_var = zips

# Creating combobox that stores zip data:
myselect = ttk.Combobox(master, values=zip_list)
myselect.current(0)
myselect.pack()

# Creating button that will print zip data and store in global variable:
mybutton = Button(master, text="Select", command=selected)
mybutton.pack()

master.mainloop()

In [ ]:
# Setting API key for RAPID API Property Snapshot API:
rapid_api_key = os.getenv('RAPID_API_KEY')

# Verifying that key was passed through accurately:
print(f"Rapid Key type: {type(rapid_api_key)}")

In [ ]:
# Connecting Rapid API to my python code which houses sales data:
conn = http.client.HTTPSConnection("realtor.p.rapidapi.com")

# Creating headers as per the Rapid API documentation:
headers = {
    'x-rapidapi-key': rapid_api_key,
    'x-rapidapi-host': "realtor.p.rapidapi.com"
    }

# Initiating API request to pull property data from Rapid API:
# Request for listing of Conodominiums within a specific zip code:
conn.request("GET", f"/properties/v2/list-for-sale?limit=5&offset=0&sort=relevance&postal_code={zip_var}",
             headers=headers)

# assigning response to variable "res" and reading the data and assigning to variable "data"_
# Also formatting data response into JSON in order to read as JSON and format:
res = conn.getresponse()
data = res.read().decode('utf-8')
json_obj = js.loads(data)
print(js.dumps(json_obj, indent=5))


In [ ]:
# Pulling data from top 5 properties to display useful sales data:
df_sale1 = pd.json_normalize(json_obj['properties'][0])
df_sale2 = pd.json_normalize(json_obj['properties'][1])
df_sale3 = pd.json_normalize(json_obj['properties'][2])
df_sale4 = pd.json_normalize(json_obj['properties'][3])
df_sale5 = pd.json_normalize(json_obj['properties'][4])

# combining sales data into single dataframe:
top_5_sale = pd.concat([df_sale1, df_sale2, df_sale3, df_sale4, df_sale5])

# Selecting columns that we want:
top_5_show = top_5_sale[['prop_type','prop_status','price','baths_full','baths','beds','thumbnail','lot_size.units',
                        'building_size.size','baths_half','mls.abbreviation']]
top_5_show

In [ ]:
# Pulling address data from top 5 properties for sale:
df_add1 = pd.json_normalize(json_obj['properties'][0]['address'])
df_add2 = pd.json_normalize(json_obj['properties'][1]['address'])
df_add3 = pd.json_normalize(json_obj['properties'][2]['address'])
df_add4 = pd.json_normalize(json_obj['properties'][3]['address'])
df_add5 = pd.json_normalize(json_obj['properties'][4]['address'])

# Updating column names for readability:
top_5_add = pd.concat([df_add1, df_add2, df_add3, df_add4, df_add5])
top_5_add.drop(columns=['fips_code'], inplace=True)
top_5_add.columns = ['City', 'Address', 'Postal Code', 'State Code', 'State', 'County','Lattitude','Longitude',
                     'Neighborhood']
top_5_add